# picodet from scratch

In [1]:
import sys
from pathlib import Path
from typing import Sequence, Optional, Union
sys.path.insert(0, "..")

import numpy as np
import torch
import torch.nn as nn
from pydantic.dataclasses import dataclass
from pydantic import Field
from mmcv.cnn import ConvModule
from mmdet.models.utils import make_divisible

from src.esnet import ESNet
from src.csppan import ChannelEqualiser, DarknetBottleneck

## Backbone: ESNet

In [2]:
esnet = ESNet()

/root/.cache/pypoetry/virtualenvs/dev-sZr6PvLR-py3.9/lib/python3.9/site-packages/mmcv/cnn/bricks/hsigmoid.py:31: UserWarning: In MMCV v1.4.4, we modified the default value of args to align with PyTorch official. Previous Implementation: Hsigmoid(x) = min(max((x + 1) / 2, 0), 1). Current Implementation: Hsigmoid(x) = min(max((x + 3) / 6, 0), 1).
  warnings.warn(


In [3]:
esnet.out_ixs

[2, 9, 12]

In [4]:
esnet.stage_out_channels
test_input = torch.from_numpy(np.random.rand(1, 3, 320, 320).astype(np.float32))
test_outputs = esnet(test_input)
print([a.shape for a in test_outputs])

[torch.Size([1, 96, 40, 40]), torch.Size([1, 192, 20, 20]), torch.Size([1, 384, 10, 10])]


In [5]:
type(test_outputs[0])

torch.Tensor

TODO: Factor inverted residual blocks into this codebase

## Neck: CSPPAN

In [6]:
c = ChannelEqualiser([96, 192, 384], 128)
channel_eq_outputs = c(test_outputs)
print([c.shape for c in channel_eq_outputs])

[torch.Size([1, 128, 40, 40]), torch.Size([1, 128, 20, 20]), torch.Size([1, 128, 10, 10])]


In [7]:
test_input = torch.from_numpy(np.random.rand(1, 32, 100, 100).astype(np.float32))
dbb = DarknetBottleneck(in_channels=32, out_channels=32)
dbb_output = dbb(test_input)
print(dbb_output.shape)

torch.Size([1, 32, 100, 100])


In [8]:
from mmcv.runner import BaseModule

In [21]:
cspl = CSPLayer(32, 32)
cspl_output = cspl(test_input)
print(cspl_output.shape)

torch.Size([1, 32, 100, 100])


In [11]:
@dataclass
class CSPPAN(BaseModule):
    in_channels: list[int]
    out_channels: int
    kernel_size: int = Field(default=5, description="Conv2D kernel size")
    n_features: int = Field(default=3, description="Number of output features of CSPPAN module")
    expansion: float = Field(default=0.5, description="...")

## Head: PicoDetHead

In [12]:
d = dict(a=5, b=6)
d.setdefault(b=8)

TypeError: dict.setdefault() takes no keyword arguments

In [ ]:
d